# Optimization in Julia

<img src="julia_opt_logo.svg" alt="Drawing" style="width: 100px;"/>

# JuliaOpt - Overview
<img src="julia_opt.svg" alt="Drawing" style="width: 750px;"/>

# Optim.jl

The Optim package provides functions for solving unconstrained/box-constrained optimization problems.

We use the Rosenbrock function to demonstrate some of its capabilities.

In [ ]:
using Optim

function f(x::Vector)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

In [ ]:
using Gadfly
plot(z=(x,y)->f([x,y]), 
     x=linspace(-0.3, 0.3, 100), y=linspace(-0.3,0.3,100), Geom.contour)

The optimize function can be used to find the minimum of Rosenbrock:

In [ ]:
start = [0.0, 0.0]
optimize(f, start)

The algorithm used to minimize can be optionally specified:

In [ ]:
res = optimize(f, [0.0, 0.0], NelderMead())

Other algorithms include: `BFGS()`, `LBFGS()`, `SimulatedAnnealing()`, `ConjugateGradient()`, `GradientDescent()`, `AcceleratedGradientDescent()`

# Optimization Options

Options are are placed in a special object with can be passed to `optimize`


In [ ]:
options = OptimizationOptions(g_tol = 1e-12, iterations = 5, store_trace = true, show_trace = true)
res = optimize(f, [0.0, 0.0], NelderMead(), options)

# Accessing results

Optim provides several functions for querying the output from an algorithm.


In [ ]:
Optim.minimum(res), Optim.minimizer(res)

Other functions include: `Optim.x_trace(res)`, `Optim.f_trace(res)`, `Optim.f_calls(res)`, `Optim.converged(res)`

# Including Gradient Information

In [ ]:
function g!(x::Vector, storage::Vector)
    storage[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    storage[2] = 200.0 * (x[2] - x[1]^2)
end

function h!(x::Vector, storage::Matrix)
    storage[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    storage[1, 2] = -400.0 * x[1]
    storage[2, 1] = -400.0 * x[1]
    storage[2, 2] = 200.0
end

In [ ]:
optimize(f, g!, h!, [0.0, 0.0], Newton())

See the Github page for more details on this package: https://github.com/JuliaOpt/Optim.jl

# JuMP.jl
JuMP (Julia Mathematical Programming) is a modelling package for constrained optimization.

In [ ]:
using JuMP

m = Model()
@variable(m, 0 <= x <= 2 )
@variable(m, 0 <= y <= 30 )

@objective(m, Max, 5x + 3*y )
@constraint(m, 1x + 5y <= 3.0 )

print(m)

In [ ]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

# Model Creation

The first step in implementing an optimization problem is to construct a model object.

In [ ]:
m = Model();

Optionally, one can specify a specific solver. Solver customization options such as maximum iterations are set in the construction of the solver object.

In [ ]:
using Clp

m = Model(solver=ClpSolver(MaximumIterations=100));

# Available Interfaces
To solve a problem, JuMP must use

<table style="text-align: center; margin-top: 20px" width="100%">
  <thead>
    <tr>
      <th width="23%"><strong>Modeling Tool</strong></th>
      <th width="9%">LP</th>
      <th width="9%">MILP</th>
      <th width="9%">SOCP</th>
      <th width="9%">MISOCP</th>
      <th width="9%">SDP</th>
      <th width="9%">NLP</th>
      <th width="9%">MINLP</th>
      <th>Other</th>
    </tr>
  </thead>

  <tfoot> <!-- Must appear before tbody, see W3.org -->
    <tr style="text-align: left">
      <td colspan="10">
        <small>
  <strong>Key:</strong>
  <ul style="list-style: none; margin-top: 5px; margin-bottom: 5px">
          <li style="margin: 0.1em 0;">LP = <a href="http://en.wikipedia.org/wiki/Linear_programming">Linear Programming</a></li>
          <li style="margin: 0.1em 0;">MILP = <a href="http://en.wikipedia.org/wiki/Integer_programming">Mixed Integer LP</a></li>
          <li style="margin: 0.1em 0;">SOCP = <a href="http://en.wikipedia.org/wiki/Second_order_cone_programming">Second-order cone programming</a> (includes convex <a href="http://en.wikipedia.org/wiki/Quadratic_programming">QP</a>, <a href="http://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program">QCQP</a>)</li>
          <li style="margin: 0.1em 0;">MISOCP = Mixed Integer SOCP</li>
          <li style="margin: 0.1em 0;">SDP = <a href="http://en.wikipedia.org/wiki/Semidefinite_programming">Semidefinite Programming</a></li>
          <li style="margin: 0.1em 0;">NLP = (constrained) <a href="http://en.wikipedia.org/wiki/Nonlinear_programming">Nonlinear Programming</a> (includes general <a href="http://en.wikipedia.org/wiki/Quadratic_programming">QP</a>, <a href="http://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program">QCQP</a>)</li>
          <li style="margin: 0.1em 0;">MINLP = Mixed Integer NLP</li>
        </ul>
        <strong>Notes:</strong>
        <ol style="">
          <li>Problem must be convex.</li>
        </ol>
  </small>
      </td>
    </tr>
  </tfoot>

  <tbody>
    <tr>
      <th><a href="https://github.com/JuliaOpt/JuMP.jl">JuMP</a></th>
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td>&#10004;</td> <!-- MISOCP -->
      <td>&#10004;</td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td>&#10004;</td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

    <tr>
      <th><a href="https://github.com/cvxgrp/Convex.jl">Convex.jl</a></th>
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td>&#10004;</td> <!-- MISOCP -->
      <td>&#10004;</td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

    <thead>
      <tr>
        <th><strong>Solver</strong></th>
        <th>LP</th>
        <th>MILP</th>
        <th>SOCP</th>
        <th>MISOCP</th>
        <th>SDP</th>
        <th>NLP</th>
        <th>MINLP</th>
        <th>Other</th>
      </tr>
    </thead>
    
    <tr>
      <th>
        <a href="https://projects.coin-or.org/Bonmin">Bonmin</a>
        <a href="https://github.com/JuliaOpt/AmplNLWriter.jl">(via AmplNLWriter.jl)</a>
      </th>

      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td>&#10004;</td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="https://projects.coin-or.org/Cbc">Cbc</a>
        <a href="https://github.com/JuliaOpt/Cbc.jl">(.jl)</a>
      </th>

      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

    <tr>
      <th>
        <a href="https://projects.coin-or.org/Clp">Clp</a>
        <a href="https://github.com/JuliaOpt/Clp.jl">(.jl)</a>
      </th>

      <td>&#10004;</td> <!-- LP -->
      <td></td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

    <tr>
      <th>
        <a href="https://projects.coin-or.org/Couenne">Couenne</a>
        <a href="https://github.com/JuliaOpt/AmplNLWriter.jl">(via AmplNLWriter.jl)</a>
      </th>

      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td>&#10004;</td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="http://www-01.ibm.com/software/commerce/optimization/cplex-optimizer/">CPLEX</a> 
        <a href="https://github.com/JuliaOpt/CPLEX.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/CPLEX.jl">CPLEX.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td>&#10004;</td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td><small>IP Callbacks</small></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="https://github.com/ifa-ethz/ecos/">ECOS</a>
        <a href="https://github.com/JuliaOpt/ECOS.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/ECOS.jl">ECOS.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td></td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="http://www.gnu.org/software/glpk/">GLPK</a>
        <a href="https://github.com/JuliaOpt/GLPK.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/GLPK.jl">GLPK.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td><small>IP Callbacks</small></td> <!-- Other -->
    </tr>

    <tr>
      <th>
        <a href="http://www.gurobi.com">Gurobi</a>
        <a href="https://github.com/JuliaOpt/Gurobi.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/Gurobi.jl">Gurobi.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td>&#10004;</td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td><small>IP Callbacks</small></td> <!-- Other -->
    </tr>

    <tr>
      <th>
        <a href="https://projects.coin-or.org/Ipopt">Ipopt</a>
        <a href="https://github.com/JuliaOpt/Ipopt.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/Ipopt.jl">Ipopt.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td></td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="http://artelys.com/en/optimization-tools/knitro">Artelys Knitro</a>
        <a href="https://github.com/JuliaOpt/KNITRO.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/KNITRO.jl/">KNITRO.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td></td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td>&#10004;</td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="http://www.mosek.com/">Mosek</a>
        <a href="https://github.com/JuliaOpt/Mosek.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/Mosek.jl">Mosek.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td>&#10004;</td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td>&#10004;</td> <!-- MISOCP -->
      <td>&#10004;</td> <!-- SDP -->
      <td>&#10004;<small>1</small></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

    <tr>
      <th>
        <a href="http://ab-initio.mit.edu/nlopt">NLopt</a>
        <a href="https://github.com/JuliaOpt/NLopt.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/NLopt.jl">NLopt.jl</a></td>-->
      <td></td> <!-- LP -->
      <td></td> <!-- MILP -->
      <td></td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td></td> <!-- SDP -->
      <td>&#10004;</td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>
    
    <tr>
      <th>
        <a href="https://github.com/cvxgrp/scs">SCS</a>
        <a href="https://github.com/JuliaOpt/SCS.jl">(.jl)</a>
      </th>
      <!--<td><a href="https://github.com/JuliaOpt/NLopt.jl">NLopt.jl</a></td>-->
      <td>&#10004;</td> <!-- LP -->
      <td></td> <!-- MILP -->
      <td>&#10004;</td> <!-- SOCP -->
      <td></td> <!-- MISOCP -->
      <td>&#10004;</td> <!-- SDP -->
      <td></td> <!-- NLP -->
      <td></td> <!-- MINLP -->
      <td></td> <!-- Other -->
    </tr>

  </tbody>
</table>


# Variables

Variables can be created individually or in blocks, and optionally with bounds. Variables, and all other problem elements are associated with a model object.

In [ ]:
@variable(m, 0 <= x[i=1:5] <= 1)

In [ ]:
@variable(m, y[i=1:5, j=1:10], Bin)

Variables over multiple indices can also be created using conditions. This is useful if you do not need to create variables over every combination of indices. For example, if you are modelling a problem on a sparse graph, you do not need to index over every pair of vertices to index over the edge set.

In [ ]:
@variable(m, z[i=1:10,j=1:10; isodd(i+j)] >= 0)

# Expressions



Constraints and Objective functions are made up of expressions. Expressions can be constructed in variety of ways.

In [ ]:
expr = x[1] + 3 * y[1,2]

The `sum` syntax is providing for convenient summing over index sets. This can be used with the expression macro.

In [ ]:
@expression(m, expr2, sum{x[i] + y[i,1], i=1:5})

In [ ]:
A = rand(5,5)
A*x # Creates an array of expressions

In [ ]:
Σ=A'A
x'Σ*x # Array of Expressions (of length 1)
dot(x, Σ*x) # Expression

# Constraints

Constraints can be constructed by combining expressions with the operators `<=`, `>=` and `==`.

In [ ]:
@constraint(m, expr <=2)

An additional argument can be included in the `@constraint` macro to a reference to the constraint object.

In [ ]:
expr4= dot(Σ*x, x)
@constraint(m, cons2, expr4 <= 5.0)

In [ ]:
@constraint(m, cons_set[j=1:10], sum{y[i, j], i=1:5} <= 4)
cons_set

# Objective Function


In [ ]:
@objective(m, Min, sum{y[i,j], i=1:5,j=1:10})

# Conic Models

Second order cone constraints can be declared using the `norm2` symbol in the constraints macro:

In [ ]:
@variable(m, t >= 0)
@constraint(m, soc, norm2{ x[i], i=1:2 } <= t)

Semidefinite variables can be declared with the `SDP` type. Positive semidefinite constraints must be added using a dedicated macro:

In [ ]:
m = Model()
@variable(m, W[1:5,1:5], SDP) # SDP type cannot have bounds
@variable(m, G[1:5,1:5] >= 0, Symmetric)

In [ ]:
J = ones(5,5)
sd_con = @SDconstraint(m, G >= J ); # SDconst has printing error in Jupyter;

# Nonlinear Models
The special macros @NLconstraint and @NLobjective are used to add nonlinear constraints and objectives.

In [ ]:
using JuMP
m = Model()
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

# adding a (linear) constraint
@constraint(m, x + y == 10)

print(m)


In [ ]:
solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

# Acknowledgements

Figures have been taken from http://juliaopt.org

Rosenbrock example taken from https://github.com/JuliaOpt/Optim.jl

Some of the JuMP snippets were taken from http://www.juliaopt.org/JuMP.jl/0.13/
